In [1]:
import ee

In [2]:
ee.Initialize(project='ee-final-year-project-2001')

In [3]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

/opt/anaconda3/envs/gee/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MCD12Q1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1

  warnings.warn(warning, category=DeprecationWarning)
/opt/anaconda3/envs/gee/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD11A1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1

  warnings.warn(warning, category=DeprecationWarning)


In [4]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [5]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

In [6]:
scale = 1000  # scale in meters

# Print the elevation near Lyon, France.
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

Ground elevation at urban point: 196 m
Average daytime LST at urban point: 23.12 °C
Land cover value at urban point is: 13


In [7]:
# Get the data for the pixel intersecting the point in urban area.
lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# Get the data for the pixel intersecting the point in rural area.
lst_r_poi = lst.getRegion(r_poi, scale).getInfo()

# Preview the result.
lst_u_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km', 'QC_Day'],
 ['2017_01_01', 4.810478346460038, 45.77365530231022, 1483228800000, None, 2],
 ['2017_01_02', 4.810478346460038, 45.77365530231022, 1483315200000, None, 2],
 ['2017_01_03', 4.810478346460038, 45.77365530231022, 1483401600000, None, 2],
 ['2017_01_04',
  4.810478346460038,
  45.77365530231022,
  1483488000000,
  13808,
  17]]

In [8]:
import geemap.core as geemap

In [28]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Create a map centered at the given coordinates with zoom level 15
m = geemap.Map(center=[40.7278, -73.9958], zoom=15)

# Load the NAIP dataset and filter by date
dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
            .filter(ee.Filter.date('2020-01-01', '2024-12-31'))

# Select the Red, Green, and Blue bands
true_color = dataset.select(['R', 'G', 'B'])

# Visualization parameters
true_color_vis = {
    'min': 0,
    'max': 255
}

# Print the first image in the collection to check
print(true_color.first().getInfo())

# Add the true color image to the map
m.add_layer(true_color.median(), true_color_vis, 'True Color')

# Display the map
m


{'type': 'Image', 'bands': [{'id': 'R', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [11830, 12310], 'crs': 'EPSG:26920', 'crs_transform': [0.6, 0, 294372, 0, -0.6, 1970706]}, {'id': 'G', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [11830, 12310], 'crs': 'EPSG:26920', 'crs_transform': [0.6, 0, 294372, 0, -0.6, 1970706]}, {'id': 'B', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [11830, 12310], 'crs': 'EPSG:26920', 'crs_transform': [0.6, 0, 294372, 0, -0.6, 1970706]}], 'version': 1688633320391069, 'id': 'USDA/NAIP/DOQQ/m_1706409_se_20_060_20220129_20220728', 'properties': {'system:time_start': 1643414400000, 'year': 2022, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-64.93936051510377, 17.74759418541972], [-64.9393597911154, 17.74759417104419], [-64.87246496311889, 17.74824438679272], [-64.87246242968055, 17.7482466418827], [-64.8724

Map(center=[40.7278, -73.9958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [25]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Create a map centered on New York City
m = geemap.Map(center=[40.7278, -73.9958], zoom=15)

# Add a satellite basemap (similar to the default JavaScript view)
m.add_basemap('SATELLITE')

# Display the map
m

Map(center=[40.7278, -73.9958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [26]:
import ee
import folium
import geemap.foliumap as geemap

# Initialize Earth Engine
ee.Initialize()

# Create a folium map
m = folium.Map(location=[40.7278, -73.9958], zoom_start=15, tiles=None)

# Add a satellite basemap layer
tile_layer = folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Satellite",
    overlay=True
)
tile_layer.add_to(m)

# Display the map
m


In [35]:
# Create the mesh
mesh = pv.PolyData(points,faces)

In [36]:
# Plot the extrusion
plotter = pv.Plotter()
plotter.add_mesh(mesh, color='blue')
plotter.show()

Widget(value='<iframe src="http://localhost:50136/index.html?ui=P_0x1556367e0_7&reconnect=auto" class="pyvista…

In [45]:
# Create a polygon (let's take a simple square as an example)
polygon = pv.Polygon(center=(0, 0, 0), radius=1.0, n_sides=4)

# Extrude the polygon to create a 3D object (like a cube)
extruded_polygon = polygon.extrude([0, 0, 1])

# Create a plotter to visualize the extruded object
plotter = pv.Plotter()
plotter.add_mesh(extruded_polygon, color="lightblue")
plotter.show()

/opt/anaconda3/envs/test/lib/python3.12/site-packages/pyvista/core/filters/poly_data.py:2997: PyVistaFutureWarning: The default value of the ``capping`` keyword argument will change in a future version to ``True`` to match the behavior of VTK. We recommend passing the keyword explicitly to prevent future surprises.
  warnings.warn(


Widget(value='<iframe src="http://localhost:50136/index.html?ui=P_0x15563b290_8&reconnect=auto" class="pyvista…